In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import requests
import threading
import time
import copy
# *rsi, *stochastic oscillator, *adx, *macd, Mass Index, ..Aroon, *ATR, Bollinger Bands, *OBV indicator

In [2]:
from sklearn import preprocessing
from sklearn.utils import shuffle

In [3]:
globalTempValueStorer = {}
globalTempValueStorer['rsi'] = {}
globalTempValueStorer['adx'] = {}
globalTempValueStorer['ema'] = {}
globalTempValueStorer['atr'] = {}
globalTempValueStorer['stoOsc'] = {}
globalTempValueStorer['slopes'] = {}
globalTempValueStorer['curves'] = {}
globalTempValueStorer['OBVind'] = {}
globalTempValueStorer['macd'] = {}
globalTempValueStorer['outProf'] = {'maxVal': 0, 'totVal': 0, 'totNum': 0}
globalTempValueStorer['ema'] = {'0': [], '1': []}
globalTempValueStorer['maxOutProfs'] = []
globalTempValueStorer['time'] = []

In [4]:
testBreakPoint = -4000

In [5]:
globalMaxLoop = 10000
globalDiff = 0.05
dataInterval = 30

In [6]:
def fetchData(start, end = '2026-01-01'):
    api_key = 'QnirZvGYSUXk9p4zerGpFqhG2paK54Ub'
    api_url = f'https://api.polygon.io/v2/aggs/ticker/X:BTCUSD/range/1/minute/{start}/{end}/?adjusted=true&sort=asc&limit=50000&apiKey={api_key}'
    raw = requests.get(api_url).json()
    data = []
    try:
        data = pd.DataFrame(raw['results'])[['t', 'o', 'h', 'l', 'c', 'v', 'vw']]
        data = data.rename(columns = {'t':'unix', 'o':'open', 'h':'high', 'l':'low', 'c':'close', 'v':'volume', 'vw':'volPrice'})
    except:
        print('Fetch did not succeed')
        time.sleep(60)
        return fetchData(start, end)
    return data

def dataSaveToCsv():
    preData = pd.read_csv('bitcoinPrices.csv')
    preData['unix'] = preData['unix'].apply(np.int64)
    preData['unix'] = preData['unix'].apply(str)
    for a in range(len(preData)-1):
        if (int(preData.iloc[a+1]['unix']) - int(preData.iloc[a]['unix']) != 60000) and (int(preData.iloc[a+1]['unix']) - int(preData.iloc[a]['unix']) != 60):
            data = fetchData(preData.iloc[a]['unix'], preData.iloc[a+1]['unix'])
            lData = len(data)
            for i in range(1, lData - 1):
                preData.loc[a+i/lData] = data.iloc[i]
    preData = preData.sort_index().reset_index(drop = True)
    start = preData.iloc[-1, 0]
    data = fetchData(start)
    returner = len(data)
    data = data.iloc[1: , :]
    data = pd.concat([preData, data], ignore_index=True)
    data.to_csv('bitcoinPrices.csv', index=False)
    data.to_csv('bitcoinSentiment.csv', index=False)
    if returner >= 49999:
        dataSaveToCsv()

def convertData(dataf, interv):
    returner = []
    for innt in range(interv):
        df = dataf[innt:]
        data = {}
        data['unix'] = df['unix'].tolist()[::interv]
        data['open'] = df['open'].tolist()[::interv]
        data['close'] = df['close'].tolist()[interv-1::interv]
        if len(data['close']) < len(data['open']):
            data['open'] = data['open'][:-1]
        high = df['high'].tolist()
        low = df['low'].tolist()
        volume = df['volume'].tolist()
        data['high'] = []
        data['low'] = []
        data['volume'] = []
        hTemp = 0.0
        lTemp = 10000000.0
        vTemp = 0.0
        for a in range(len(high)):
            hTemp = max(hTemp, high[a])
            lTemp = min(lTemp, low[a])
            vTemp += volume[a]
            if (a+1)%interv == 0:
                data['high'].append(hTemp)
                data['low'].append(lTemp)
                data['volume'].append(vTemp)
                hTemp = 0.0
                lTemp = 10000000.0
                vTemp = 0.0
        returner.append(data)
    return returner

def checkData(dataset):
    global globalTempValueStorer
    prev = 0
    q = dataset['unix'].tolist()
    for i in q:
        if prev == 0:
            prev = int(str(i)[:9])
        else:
            pr = prev
            prev = int(str(i)[:9])
            if prev - pr != 6:
                print('Data problem! at unix', prev)
    globalTempValueStorer['lastUnixProcessed'] = int(q[-1])
    print('helloo', globalTempValueStorer['lastUnixProcessed'])
    print('Data check complete!, last time point:', q[-1])

In [7]:
def getDfAndDataset(dataInterval = dataInterval):
    global dataset, df
    dataSaveToCsv()
    dataset = pd.read_csv('bitcoinPrices.csv')
    checkData(dataset)
    df = convertData(dataset, dataInterval)

In [8]:
dataset, df = [], []
getDfAndDataset()

Data problem! at unix 168356376
Data problem! at unix 168359040
helloo 1683676680000
Data check complete!, last time point: 1683676680000


In [9]:
%run ./twitterSentiment.ipynb

In [10]:
sentimentData = pd.read_csv('bitcoinSentiment.csv')

In [11]:
sentimentPositive = []
sentimentNegative = []
for i in range(dataInterval):
    sentimentPositive.append(sentimentData['tweetSentimentPositive'].tolist()[i+dataInterval-1::dataInterval])
    sentimentNegative.append(sentimentData['tweetSentimentNegative'].tolist()[i+dataInterval-1::dataInterval])

In [12]:
globalTempValueStorer['dataStraight'] = {'unix': dataset['unix'].to_list(), 'open': dataset['open'].to_list(), 'high': dataset['high'].to_list(), 'close': dataset['close'].to_list(), 'low': dataset['low'].to_list(), 'volume': dataset['volume'].to_list()}

In [13]:
# distance from EMA to be implemented
def findNd(arr):
    nd = 0
    t = len(arr[0])
    for i in range(1, len(arr)):
        if len(arr[i]) < t:
            nd = i
            break
    return nd

def EMA(df, interv, series=0, EMAArray = []):
    def InitialCalc(df, interv, series):
        global globalTempValueStorer
        globalTempValueStorer['ema'][series] = {}
        emaRet = []
        multiplier = 2.0/(interv + 1.0)
        for innt in range(len(df)):
            data = df[innt]
            ema = ['NA' for a in range(interv-1)]
            EMA = 0.0
            for a in range(interv):
                EMA += data['close'][a]
            EMA /= interv
            ema.append(EMA)
            for a in range(len(data['close']) - interv):
                EMA = (1 - multiplier) * EMA + data['close'][a + interv] * multiplier
                ema.append(EMA)
            emaRet.append(ema)
            globalTempValueStorer['ema'][series][innt] = EMA
        return emaRet
    
    #Update needs close of closing candle
    def Update(df, interv, series, EMAArray):
        global globalTempValueStorer
        nd = findNd(EMAArray)
        multiplier = 2.0/(interv + 1.0)
        globalTempValueStorer['ema'][series][nd] = (1 - multiplier) * globalTempValueStorer['ema'][series][nd] + df['close'] * multiplier
        EMAArray[nd].append(globalTempValueStorer['ema'][series][nd])
        return globalTempValueStorer['ema'][series][nd]
    
    if EMAArray == []:
        return InitialCalc(df, interv, series)
    else:
        return Update(df, interv, series, EMAArray)

def StoOsc(df, interv, series = 0, StoOscArray = []):
    def InitialCalc(df, interv, series):
        global globalTempValueStorer
        globalTempValueStorer['stoOsc'][series] = {}
        soRet = []
        for innt in range(len(df)):
            data = df[innt]
            so = ['NA' for a in range(interv-1)]
            for a in range(len(data['close']) - interv + 1):
                so.append(100*(data['close'][a] - min(data['close'][a:a+interv]))/(max(data['close'][a:a+interv]) - min(data['close'][a:a+interv])))
            globalTempValueStorer['stoOsc'][series][innt] = data['close'][-interv + 1:]
            soRet.append(so)
        return soRet

    #Update needs close of closing candle
    def Update(df, interv, series, StoOscArray):
        global globalTempValueStorer
        nd = findNd(StoOscArray)
        globalTempValueStorer['stoOsc'][series][nd].append(df['close'])
        temp = globalTempValueStorer['stoOsc'][series][nd]
        StoOscArray[nd].append(100*(temp[0] - min(temp))/(max(temp) - min(temp)))
        globalTempValueStorer['stoOsc'][series][nd].pop(0)
        return StoOscArray[nd][-1]
    
    if StoOscArray == []:
        return InitialCalc(df, interv, series)
    else:
        return Update(df, interv, series, StoOscArray)
        

def ATR(df, interv, series = 0, ATRArray = []):
    def InitialCalc(df, interv, series):
        global globalTempValueStorer
        globalTempValueStorer['atr'][series] = {}
        retAtr = []
        for innt in range(len(df)):
            data = df[innt]
            Atr = ['NA' for a in range(interv-1)]
            atrTemp = 0.0
            for a in range(interv):
                if a == 0:
                    atrTemp += data['high'][a] - data['low'][a]
                else:
                    atrTemp += max(data['high'][a] - data['low'][a], abs(data['high'][a] - data['close'][a - 1]), abs(data['close'][a - 1] - data['low'][a]))
            atrTemp /= float(interv)
            Atr.append(atrTemp)
            for a in range(len(data['high']) - interv):
                atrTemp = ((atrTemp*(interv - 1) + max(data['high'][a + interv] - data['low'][a + interv], abs(data['high'][a + interv] - data['close'][a  + interv - 1]), abs(data['close'][a  + interv - 1] - data['low'][a + interv]))))/interv
                Atr.append(atrTemp)
            retAtr.append(Atr)
            globalTempValueStorer['atr'][series][innt] = {'close': data['close'][-1], 'atrTemp': atrTemp}
        return retAtr
    
    #Update needs high, low and close of closing candle
    def Update(df, interv, series, ATRArray):
        global globalTempValueStorer
        nd = findNd(ATRArray)
        atrTemp = globalTempValueStorer['atr'][series][nd]['atrTemp']
        cl = globalTempValueStorer['atr'][series][nd]['close']
        atrTemp = ((atrTemp*(interv - 1) + max(df['high'] - df['low'], abs(df['high'] - cl), abs(cl - df['low']))))/interv
        ATRArray[nd].append(atrTemp)
        globalTempValueStorer['atr'][series][nd] = {'close': df['close'], 'atrTemp': atrTemp}
        return atrTemp
    
    if ATRArray == []:
        return InitialCalc(df, interv, series)
    else:
        return Update(df, interv, series, ATRArray)

def ADX(df, interv, series = 0, ADXArray = []):
    def InitialCalc(df, interv, series):
        global globalTempValueStorer
        globalTempValueStorer['adx'][series] = {}
        adxRet = []
        multiplier = 2.0/(interv + 1.0)
        for innt in range(len(df)):
            data = df[innt]
            plusDM = ['NA']
            minusDM = ['NA']
            for a in range(len(data['high']) - 1):
                pdm = 0.0
                mdm = 0.0
                UM = data['high'][a+1] - data['high'][a]
                DM = data['low'][a] - data['low'][a+1]
                if UM > DM and UM > 0:
                    pdm = UM
                elif DM > UM and DM > 0:
                    mdm = DM
                plusDM.append(pdm)
                minusDM.append(mdm)
            adx = ['NA' for a in range(interv)]
            pdiTemp = sum(plusDM[1:interv+1])/interv
            mdiTemp = sum(minusDM[1:interv+1])/interv
            adx.append(100*abs(pdiTemp - mdiTemp)/(pdiTemp + mdiTemp))
            for a in range(len(data['high']) - interv - 1):
                pdiTemp = plusDM[a + interv + 1]*multiplier + pdiTemp*(1-multiplier)
                mdiTemp = minusDM[a + interv + 1]*multiplier + mdiTemp*(1-multiplier)
                adx.append(100*abs(pdiTemp - mdiTemp)/(pdiTemp + mdiTemp))
            globalTempValueStorer['adx'][series][innt] = {'pdiTemp': pdiTemp, 'mdiTemp': mdiTemp, 'lastDataHigh': data['high'][-1], 'lastDataLow': data['low'][-1]}
            adxRet.append(adx)
        return adxRet
    
    #Update needs high and low of closing candle
    def Update(df, interv, series, ADXArray):
        global globalTempValueStorer
        multiplier = 2.0/(interv + 1.0)
        nd = findNd(ADXArray)
        currTemp = globalTempValueStorer['adx'][series][nd]
        pdm = 0.0
        mdm = 0.0
        UM = df['high'] - currTemp['lastDataHigh']
        DM = currTemp['lastDataLow'] - df['low']
        if UM > DM and UM > 0:
            pdm = UM
        elif DM > UM and DM > 0:
            mdm = DM
        pdiTemp = pdm*multiplier + currTemp['pdiTemp']*(1-multiplier)
        mdiTemp = mdm*multiplier + currTemp['mdiTemp']*(1-multiplier)
        ADXArray[nd].append(100*abs(pdiTemp - mdiTemp)/(pdiTemp + mdiTemp))
        globalTempValueStorer['adx'][series][nd] = {'pdiTemp': pdiTemp, 'mdiTemp': mdiTemp, 'lastDataHigh': df['high'], 'lastDataLow': df['low']}
        return ADXArray[nd][-1]
        
    if ADXArray == []:
        return InitialCalc(df, interv, series)
    else:
        return Update(df, interv, series, ADXArray)

def OBVind(df, OBVindArray = []):
    def InitialCalc(df):
        global globalTempValueStorer
        globalTempValueStorer['OBVind'] = {}
        indRet = []
        indProcess = []
        for innt in range(len(df)):
            data = df[innt]
            obvInd = []
            obvInd2 = []
            obvTemp = 100000
            obvTemp2 = 0
            for a in range(len(data['close'])):
                if data['close'][a] - data['open'][a] != 0:
                    obvTemp += data['volume'][a]/(data['close'][a] - data['open'][a])
                    obvTemp2 = data['volume'][a]/(data['close'][a] - data['open'][a])
                else:
                    obvTemp2 = 0
                obvInd.append(obvTemp)
                obvInd2.append(obvTemp2)
            globalTempValueStorer['OBVind'][innt] = {'obvInd': obvInd, 'obvTemp': obvTemp}
            indRet.append(obvInd2)
            indProcess.append({'open': [obvInd[0]] + obvInd[:-1], 'high': [max([[obvInd[0]] + obvInd[:-1]][0][a], obvInd[a]) for a in range(len(obvInd))], 'low': [min([[obvInd[0]] + obvInd[:-1]][0][a], obvInd[a]) for a in range(len(obvInd))], 'close': obvInd})
        return [indRet, indProcess]
    
    #Update needs volume, close and open of closing candle
    def Update(df, OBVindArray):
        global globalTempValueStorer
        nd = findNd(OBVindArray[0])
        obvTemp = globalTempValueStorer['OBVind'][nd]['obvTemp']
        obvInd = globalTempValueStorer['OBVind'][nd]['obvInd']
        obvTemp2 = 0
        if df['close'] - df['open'] != 0:
            obvTemp += df['volume']/(df['close'] - df['open'])
            obvTemp2 = df['volume']/(df['close'] - df['open'])
        obvInd.append(obvTemp)
        globalTempValueStorer['OBVind'][nd] = {'obvInd': obvInd, 'obvTemp': obvTemp}
        OBVindArray[0][nd].append(obvTemp2)
        OBVindArray[1][nd] = {'open': [obvInd[0]] + obvInd[:-1], 'high': [max([[obvInd[0]] + obvInd[:-1]][0][a], obvInd[a]) for a in range(len(obvInd))], 'low': [min([[obvInd[0]] + obvInd[:-1]][0][a], obvInd[a]) for a in range(len(obvInd))], 'close': obvInd}
        return [OBVindArray[0][nd][-1], 
                {
                    'open': OBVindArray[1][nd]['open'][-1],
                    'high': OBVindArray[1][nd]['high'][-1],
                    'low': OBVindArray[1][nd]['low'][-1],
                    'close': OBVindArray[1][nd]['close'][-1],
                }]
    
    if OBVindArray == []:
        return InitialCalc(df)
    else:
        return Update(df, OBVindArray)

def RSI(df, interv, series = 0, RSIArray = []):
    def InitialCalc(df, interv, series):
        global globalTempValueStorer
        globalTempValueStorer['rsi'][series] = {}
        rsiRet = []
        for innt in range(len(df)):
            data = df[innt]
            rsi = ['NA' for a in range(interv-1)]
            prProf = 0.0
            prLoss = 0.0
            for a in range(interv):
                t = data['open'][a] - data['close'][a]
                if t > 0:
                    prLoss += float(t)
                else:
                    prProf -= float(t)
            rsi.append((100.0-100.0/(1 + (prProf/prLoss))))
            prProf = prProf/float(interv)
            prLoss = prLoss/float(interv)
            for a in range(len(data['open']) - interv):
                temp = data['open'][a + interv] - data['close'][a + interv]
                p = 0.0
                l = 0.0
                if temp > 0:
                    l = temp
                else:
                    p = -temp
                prProf = (prProf*float(interv-1) + p)/float(interv)
                prLoss = (prLoss*float(interv-1) + l)/float(interv)
                rsi.append((100.0*prProf/(prProf + prLoss)))
            rsiRet.append(rsi)
            globalTempValueStorer['rsi'][series][innt] = {'prProf': prProf, 'prLoss': prLoss}
        return rsiRet
    
    #Update needs open and close of closing candle
    def Update(df, interv, series, RSIArray):
        global globalTempValueStorer
        nd = findNd(RSIArray)
        temp = df['open'] - df['close']
        p = 0.0
        l = 0.0
        if temp > 0:
            l = temp
        else:
            p = -temp
        prProf = globalTempValueStorer['rsi'][series][nd]['prProf']
        prLoss = globalTempValueStorer['rsi'][series][nd]['prLoss']
        globalTempValueStorer['rsi'][series][nd] = {'prProf': (prProf*float(interv-1) + p)/float(interv), 'prLoss': (prLoss*float(interv-1) + l)/float(interv)}
        temp = 100.0*globalTempValueStorer['rsi'][series][nd]['prProf']/(globalTempValueStorer['rsi'][series][nd]['prProf'] + globalTempValueStorer['rsi'][series][nd]['prLoss'])
        RSIArray[nd].append(temp)
        globalTempValueStorer['blahhhhh'] = [nd, temp, RSIArray[nd][-2]]
        return temp
        
    if RSIArray == []:
        return InitialCalc(df, interv, series)
    else:
        return Update(df, interv, series, RSIArray)
                
def MACD(df, intervS, intervL, series=0, MACDArray = []):
    def InitialCalc(df, intervS, intervL, series):
        global globalTempValueStorer
        globalTempValueStorer['macd'][series] = {}
        macdRet = []
        emaShort = EMA(df, intervS, str(series) + ' MACD ' + str(intervS))
        emaLong = EMA(df, intervL, str(series) + ' MACD ' + str(intervL))
        for innt in range(len(df)):
            data = df[innt]
            emaS = emaShort[innt]
            emaL = emaLong[innt]
            temp = [emaS[a] - emaL[a] for a in range(intervL - 1, len(data['high']))]
            macd = ['NA' for a in range(intervL - 1)] + temp
            macdRet.append(macd)
        globalTempValueStorer['macd'][series]['emaShort'] = emaShort
        globalTempValueStorer['macd'][series]['emaLong'] = emaLong
        return macdRet
    
    #Update needs close of closing candle
    def Update(df, intervS, intervL, series, MACDArray):
        global globalTempValueStorer
        nd = findNd(MACDArray)
        emaSU = EMA(df, intervS, str(series) + ' MACD ' + str(intervS), globalTempValueStorer['macd'][series]['emaShort'])
        emaLU = EMA(df, intervL, str(series) + ' MACD ' + str(intervS), globalTempValueStorer['macd'][series]['emaLong'])
        MACDArray[nd].append(emaSU - emaLU)
        return MACDArray[nd][-1]
        
    if MACDArray == []:
        return InitialCalc(df, intervS, intervL, series)
    else:
        return Update(df, intervS, intervL, series, MACDArray)

def slopeInd(df, indSet = '', series = '', slopeArray = []):
    def InitialCalc(df, indSet, series):
        global globalTempValueStorer
        globalTempValueStorer['slopes'][str(indSet) + ' ' + str(series)] = {}
        sRet = []
        for innt in range(len(df)):
            ind = df[innt]
            for a in range(len(ind)):
                if not isinstance(ind[a], str):
                    break
            indSlopes = ['NA' for b in range(a + 1)]
            for b in range(len(ind) - a - 1):
                indSlopes.append(ind[b + a + 1] - ind[b + a])
            sRet.append(indSlopes)
            globalTempValueStorer['slopes'][str(indSet) + ' ' + str(series)][innt] = ind[-1]
        return sRet
    
    #Update needs current value of Indicator
    def Update(df, indSet, series, slopeArray):
        global globalTempValueStorer
        nd = findNd(slopeArray)
        slopeArray[nd].append(df - globalTempValueStorer['slopes'][str(indSet) + ' ' + str(series)][nd])
        globalTempValueStorer['slopes'][str(indSet) + ' ' + str(series)][nd] = df
        return slopeArray[nd][-1]
    
    if slopeArray == []:
        return InitialCalc(df, indSet, series)
    else:
        return Update(df, indSet, series, slopeArray)

def curevInd(df, indSet = '', series = '', curveArray = []):
    def InitialCalc(df, indSet, series):
        global globalTempValueStorer
        globalTempValueStorer['curves'][str(indSet) + ' ' + str(series)] = {}
        sRet = []
        for innt in range(len(df)):
            ind = df[innt]
            for a in range(len(ind)):
                if not isinstance(ind[a], str):
                    break
            indSlopes = ['NA' for b in range(a + 3)]
            for b in range(len(ind) - a - 3):
                indSlopes.append((2.0*ind[b + a + 3] - 3.0*ind[b + a + 2] + ind[b + a])/2.0)
            sRet.append(indSlopes)
            globalTempValueStorer['curves'][str(indSet) + ' ' + str(series)][innt] = [ind[-3], ind[-2], ind[-1]]
        return sRet
    
    #Update needs current value of Indicator
    def Update(df, indSet, series, curveArray):
        global globalTempValueStorer
        nd = findNd(curveArray)
        temp = globalTempValueStorer['curves'][str(indSet) + ' ' + str(series)][nd]
        curveArray[nd].append((2.0*df - 3.0*temp[-1] + temp[-3])/2.0)
        globalTempValueStorer['curves'][str(indSet) + ' ' + str(series)][nd].append(df)
        globalTempValueStorer['curves'][str(indSet) + ' ' + str(series)][nd].pop(0)
        return curveArray[nd][-1]
    
    if curveArray == []:
        return InitialCalc(df, indSet, series)
    else:
        return Update(df, indSet, series, curveArray)

In [14]:
def OutHighLow(df, interv):
    interv += 1
    oHL = {}
    oHL['high'] = []
    oHL['low'] = []
    for innt in range(len(df)):
        data = df[innt]
        outHL = {}
        outHL['high'] = []
        outHL['low'] = []
        for a in range(len(data['close']) - interv + 1):
            outHL['high'].append(100.0*(max(data['high'][a + 1 : a + interv]) - data['close'][a])/data['close'][a])
            outHL['low'].append(100.0*(min(data['low'][a + 1 : a + interv]) - data['close'][a])/data['close'][a])
        outHL['high'] += ['NA' for a in range(interv - 1)]
        outHL['low'] +=  ['NA' for a in range(interv - 1)]
        oHL['high'].append(outHL['high'])
        oHL['low'].append(outHL['low'])
    return oHL

def OutProf(incH, incL, df = df, maxLoop = globalMaxLoop, diff = globalDiff):
    global globalTempValueStorer
    global testBreakPoint
    global dataInterval
    oHL = []
    returns = []
    subb = float(incH)/float(incH + incL)
    lengthDf = len(df)
    lenDataStraight = len(globalTempValueStorer['dataStraight']['close'])
    for innt in range(lengthDf):
        print(innt, 'out of', lengthDf)
        retVal = []
        data = df[innt]
        ind = []
        for a in range(len(data['close'])):
            numDataCurrent = a*lengthDf + innt
            aa = lengthDf
            temp = 'NA'
            outH = 0
            outL = 0
            numLoop = 0
            thisDataClose = data['close'][a]
            if numDataCurrent + aa < lenDataStraight:
                rets = [[0, [data['unix'][a], thisDataClose], [globalTempValueStorer['dataStraight']['unix'][numDataCurrent + aa], globalTempValueStorer['dataStraight']['close'][numDataCurrent + aa], globalTempValueStorer['dataStraight']['high'][numDataCurrent + aa], globalTempValueStorer['dataStraight']['low'][numDataCurrent + aa]]]]
            else:
                rets = [[0, [data['unix'][a], thisDataClose], [data['unix'][a], data['close'][a], data['high'][a], data['low'][a]]]]
            while numDataCurrent + aa < lenDataStraight and numLoop < maxLoop:
                outH = max(outH, 100.0*(globalTempValueStorer['dataStraight']['high'][numDataCurrent + aa] - thisDataClose)/thisDataClose)
                outL = max(outL, -100.0*(globalTempValueStorer['dataStraight']['low'][numDataCurrent + aa] - thisDataClose)/thisDataClose)
                #toStore = {'timestamp': globalTempValueStorer['dataStraight']['unix'][numDataCurrent + aa], 'open': globalTempValueStorer['dataStraight']['open'][numDataCurrent + aa], 'high': globalTempValueStorer['dataStraight']['high'][numDataCurrent + aa], 'low': globalTempValueStorer['dataStraight']['low'][numDataCurrent + aa], 'close': globalTempValueStorer['dataStraight']['close'][numDataCurrent + aa]}
                if outH > incH and outL > incL:
                    if outH >= outL:
                        if (incH + incL) <= incH:
                            temp = 1 + diff
                            #globalTempValueStorer['maxOutProfs'].append([0, a, innt, data['unix'][a], outH, outL, incH, incL, numDataCurrent, thisDataClose, toStore])
                        else:
                            temp = diff + (incH/(incH + incL) - subb)/(1 - subb)
                    else:
                        if outL >= (incH + incL):
                            temp = -1 - diff
                            #globalTempValueStorer['maxOutProfs'].append([1, a, innt, data['unix'][a], outH, outL, incH, incL, numDataCurrent, thisDataClose, toStore])
                        else:
                            temp = (1 - incL/(incH + incL) - subb)/subb - diff
                    break
                elif outH > incH:
                    if (incH + outL) <= incH:
                        temp = 1 + diff
                        #globalTempValueStorer['maxOutProfs'].append([2, a, innt, data['unix'][a], outH, outL, incH, incL, numDataCurrent, thisDataClose, toStore])
                    else:
                        temp = diff + (incH/(incH + outL) - subb)/(1 - subb)
                    break
                elif outL > incL:
                    if incL >= (outH + incL):
                        temp = -1 - diff
                        #globalTempValueStorer['maxOutProfs'].append([3, a, innt, data['unix'][a], outH, outL, incH, incL, numDataCurrent, thisDataClose, toStore])
                    else:
                        temp = (1 - incL/(outH + incL) - subb)/subb - diff
                    break
                aa += 1
                numLoop += 1
            globalTempValueStorer['outProf']['maxVal'] = max(globalTempValueStorer['outProf']['maxVal'], numLoop)
            globalTempValueStorer['outProf']['totNum'] += 1
            globalTempValueStorer['outProf']['totVal'] += numLoop
            if isinstance(temp, str):
                globalTempValueStorer['notCompleteOutputs'] += 1
                if len(rets) > 1:
                    if rets[-1][0] > 0:
                        temp = 0.5
                    elif rets[-1][0] < 0:
                        temp = -0.5
                    else:
                        temp = 0
                else:
                    temp = 0
            else:
                globalTempValueStorer['completeOutputs'] += 1
                if numDataCurrent >= lenDataStraight + testBreakPoint*lengthDf:
                    rets.append([100.0*(globalTempValueStorer['dataStraight']['close'][numDataCurrent + aa] - thisDataClose)/thisDataClose, [data['unix'][a], thisDataClose], [globalTempValueStorer['dataStraight']['unix'][numDataCurrent + aa], globalTempValueStorer['dataStraight']['close'][numDataCurrent + aa], globalTempValueStorer['dataStraight']['high'][numDataCurrent + aa], globalTempValueStorer['dataStraight']['low'][numDataCurrent + aa]]])
            retVal.append(rets)
            ind.append(temp)
        returns.append(retVal)
        oHL.append(ind)
        
    return [oHL, returns]

def InputOutputGiver(trainArrays, slicer, takeTime = False, df = df):
    retIn = [[] for a in range(len(trainArrays))]
    globalTempValueStorer['time'] = []
    
    temp0, temp1 = slicer[0], slicer[1]
    if slicer[0] < 0:
        temp0 = len(trainArrays[0][0]) + slicer[0]
    if slicer[1] < 0:
        temp1 = len(trainArrays[0][0]) + slicer[1]
    
    for a in range(len(trainArrays)):
        for c in range(temp0, temp1):
            for b in range(len(trainArrays[a])):
                try:
                    retIn[a].append(trainArrays[a][b][c])
                except:
                    print(len(trainArrays[a][b]), c)
                    print(trainArrays[a][b][c])
    
    if takeTime:
        for c in range(temp0, temp1):
            for b in range(len(trainArrays[0])):
                globalTempValueStorer['time'] += [df[b]['unix'][c]]
    return np.array(retIn).T

In [15]:
outputs = [0.0 for a in range(10)]
outReturns = [0.0 for a in range(10)]

In [16]:
# max in a direction --> 2.25

outputS = [0 for a in range(10)]
outReturns = [0 for a in range(10)]

globalTempValueStorer['notCompleteOutputs'] = 0
globalTempValueStorer['completeOutputs'] = 0
hlPair = [
    [3.25, 2.25],
    [3.0, 2.15],
    [2.75, 2.0],
    [2.5, 1.7],
    [2.0, 1.3]
]
for i in range(len(hlPair)):
    [outputS[2*i], outReturns[2*i]] = OutProf(hlPair[i][0], hlPair[i][1])
    print(i, 'pair upside complete')
    [outputS[2*i+1], outReturns[2*i+1]] = OutProf(hlPair[i][1], hlPair[i][0])
    print(i, 'pair downside complete')

0 out of 30
1 out of 30
2 out of 30
3 out of 30
4 out of 30
5 out of 30
6 out of 30
7 out of 30
8 out of 30
9 out of 30
10 out of 30
11 out of 30
12 out of 30
13 out of 30
14 out of 30
15 out of 30
16 out of 30
17 out of 30
18 out of 30
19 out of 30
20 out of 30
21 out of 30
22 out of 30
23 out of 30
24 out of 30
25 out of 30
26 out of 30
27 out of 30
28 out of 30
29 out of 30
0 pair upside complete
0 out of 30
1 out of 30
2 out of 30
3 out of 30
4 out of 30
5 out of 30
6 out of 30
7 out of 30
8 out of 30
9 out of 30
10 out of 30
11 out of 30
12 out of 30
13 out of 30
14 out of 30
15 out of 30
16 out of 30
17 out of 30
18 out of 30
19 out of 30
20 out of 30
21 out of 30
22 out of 30
23 out of 30
24 out of 30
25 out of 30
26 out of 30
27 out of 30
28 out of 30
29 out of 30
0 pair downside complete
0 out of 30
1 out of 30
2 out of 30
3 out of 30
4 out of 30
5 out of 30
6 out of 30
7 out of 30
8 out of 30
9 out of 30
10 out of 30
11 out of 30
12 out of 30
13 out of 30
14 out of 30
15 out 

In [17]:
print(globalTempValueStorer['notCompleteOutputs'], globalTempValueStorer['completeOutputs'])

102039 14225391


In [18]:
globalTempValueStorer['outProfReturnStraight'] = []
for a in range(len(outReturns)):
    outsw = []
    for b in range(len(outReturns[a][0])):
        for c in range(len(outReturns[a])):
            if b < len(outReturns[a][c]):
                outsw.append(outReturns[a][c][b])
    globalTempValueStorer['outProfReturnStraight'].append(outsw)

In [19]:
def fillInputs(df, tfInput, updateArr = []):
    tfInput['rsi'] = RSI(df, 14, 0, [] if updateArr == [] else updateArr['rsi'])
    tfInput['rsiSlope'] = slopeInd(tfInput['rsi'], 'nrsi', 0, [] if updateArr == [] else updateArr['rsiSlope'])
    tfInput['rsiCurve'] = curevInd(tfInput['rsi'], 'nrsi', 0, [] if updateArr == [] else updateArr['rsiCurve'])
    tfInput['rsi2'] = RSI(df, 40, 1, [] if updateArr == [] else updateArr['rsi2'])
    tfInput['rsi2Slope'] = slopeInd(tfInput['rsi2'], 'nrsi', 1, [] if updateArr == [] else updateArr['rsi2Slope'])
    tfInput['rsi2Curve'] = curevInd(tfInput['rsi2'], 'nrsi', 1, [] if updateArr == [] else updateArr['rsi2Curve'])
    tfInput['ema'] = EMA(df, 50, 0, [] if updateArr == [] else updateArr['ema'])
    tfInput['macd'] = MACD(df, 14, 28, 0, [] if updateArr == [] else updateArr['macd'])
    tfInput['macd2'] = MACD(df, 30, 70, 1, [] if updateArr == [] else updateArr['macd2'])
    tfInput['macd3'] = MACD(df, 75, 150, 2, [] if updateArr == [] else updateArr['macd3'])
    tfInput['macdSlope'] = slopeInd(tfInput['macd'], 'nmacd', 0, [] if updateArr == [] else updateArr['macdSlope'])
    tfInput['macdCurve'] = curevInd(tfInput['macd'], 'nmacd', 0, [] if updateArr == [] else updateArr['macdCurve'])
    tfInput['macd2Slope'] = slopeInd(tfInput['macd2'], 'nmacd', 1, [] if updateArr == [] else updateArr['macd2Slope'])
    tfInput['macd2Curve'] = curevInd(tfInput['macd2'], 'nmacd', 1, [] if updateArr == [] else updateArr['macd2Curve'])
    tfInput['macd3Slope'] = slopeInd(tfInput['macd3'], 'nmacd', 2, [] if updateArr == [] else updateArr['macd3Slope'])
    tfInput['macd3Curve'] = curevInd(tfInput['macd3'], 'nmacd', 2, [] if updateArr == [] else updateArr['macd3Curve'])
    tfInput['adx'] = ADX(df, 25, 0, [] if updateArr == [] else updateArr['adx'])
    tfInput['adxSlope'] = slopeInd(tfInput['adx'], 'nadx', 0, [] if updateArr == [] else updateArr['adxSlope'])
    tfInput['adxCurve'] = curevInd(tfInput['adx'], 'nadx', 0, [] if updateArr == [] else updateArr['adxCurve'])
    tfInput['stOs'] = StoOsc(df, 14, 0, [] if updateArr == [] else updateArr['stOs'])
    tfInput['stOsSlope'] = slopeInd(tfInput['stOs'], 'nstOs', 0, [] if updateArr == [] else updateArr['stOsSlope'])
    tfInput['stOsCurve'] = curevInd(tfInput['stOs'], 'nstOs', 0, [] if updateArr == [] else updateArr['stOsCurve'])
    tfInput['stOs2'] = StoOsc(df, 28, 1, [] if updateArr == [] else updateArr['stOs2'])
    tfInput['stOs2Slope'] = slopeInd(tfInput['stOs2'], 'nstOs', 1, [] if updateArr == [] else updateArr['stOs2Slope'])
    tfInput['stOs2Curve'] = curevInd(tfInput['stOs2'], 'nstOs', 1, [] if updateArr == [] else updateArr['stOs2Curve'])
    tfInput['stOs3'] = StoOsc(df, 50, 2, [] if updateArr == [] else updateArr['stOs3'])
    tfInput['stOs3Slope'] = slopeInd(tfInput['stOs3'], 'nstOs', 2, [] if updateArr == [] else updateArr['stOs3Slope'])
    tfInput['stOs3Curve'] = curevInd(tfInput['stOs3'], 'nstOs', 2, [] if updateArr == [] else updateArr['stOs3Curve'])
    tfInput['Obv'] = OBVind(df, [] if updateArr == [] else updateArr['Obv'])
    tfInput['Oind'] = tfInput['Obv'][0]
    tfInput['OindSlope'] = slopeInd(tfInput['Oind'], 'oind', 0, [] if updateArr == [] else updateArr['OindSlope'])
    tfInput['OindCurve'] = curevInd(tfInput['Oind'], 'oind', 0, [] if updateArr == [] else updateArr['OindCurve'])
    tfInput['OindMacd'] = MACD(tfInput['Obv'][1], 7, 28, 3, [] if updateArr == [] else updateArr['OindMacd'])
    tfInput['OindMacdSlope'] = slopeInd(tfInput['OindMacd'], 'omacd', 3, [] if updateArr == [] else updateArr['OindMacdSlope'])
    tfInput['OindMacdCurve'] = curevInd(tfInput['OindMacd'], 'omacd', 3, [] if updateArr == [] else updateArr['OindMacdCurve'])
    tfInput['OindRsi'] = RSI(tfInput['Obv'][1], 14, 2, [] if updateArr == [] else updateArr['OindRsi'])
    tfInput['OindRsiSlope'] = slopeInd(tfInput['OindRsi'], 'orsi', 2, [] if updateArr == [] else updateArr['OindRsiSlope'])
    tfInput['OindRsiCurve'] = curevInd(tfInput['OindRsi'], 'orsi', 2, [] if updateArr == [] else updateArr['OindRsiCurve'])
    tfInput['OindMacd2'] = MACD(tfInput['Obv'][1], 35, 100, 4, [] if updateArr == [] else updateArr['OindMacd2'])
    tfInput['OindMacd2Slope'] = slopeInd(tfInput['OindMacd2'], 'omacd', 4, [] if updateArr == [] else updateArr['OindMacd2Slope'])
    tfInput['OindMacd2Curve'] = curevInd(tfInput['OindMacd2'], 'omacd', 4, [] if updateArr == [] else updateArr['OindMacd2Curve'])
    tfInput['OindRsi2'] = RSI(tfInput['Obv'][1], 40, 3, [] if updateArr == [] else updateArr['OindRsi2'])
    tfInput['OindRsi2Slope'] = slopeInd(tfInput['OindRsi2'], 'orsi', 3, [] if updateArr == [] else updateArr['OindRsi2Slope'])
    tfInput['OindRsi2Curve'] = curevInd(tfInput['OindRsi2'], 'orsi', 3, [] if updateArr == [] else updateArr['OindRsi2Curve'])
    tfInput['OindstOs'] = StoOsc(tfInput['Obv'][1], 14, 3, [] if updateArr == [] else updateArr['OindstOs'])
    tfInput['OindstOsSlope'] = slopeInd(tfInput['OindstOs'], 'ostosc', 4, [] if updateArr == [] else updateArr['OindstOsSlope'])
    tfInput['OindstOsCurve'] = curevInd(tfInput['OindstOs'], 'ostosc', 4, [] if updateArr == [] else updateArr['OindstOsCurve'])
    tfInput['OindAdx'] = ADX(tfInput['Obv'][1], 25, 1, [] if updateArr == [] else updateArr['OindAdx'])
    tfInput['OindAdxSlope'] = slopeInd(tfInput['OindAdx'], 'oadx', 4, [] if updateArr == [] else updateArr['OindAdxSlope'])
    tfInput['OindAdxCurve'] = curevInd(tfInput['OindAdx'], 'oadx', 4, [] if updateArr == [] else updateArr['OindAdxCurve'])
    tfInput['atr'] = ATR(df, 14, 0, [] if updateArr == [] else updateArr['atr'])
    tfInput['atrSlope'] = slopeInd(tfInput['atr'], 'natr', 0, [] if updateArr == [] else updateArr['atrSlope'])
    tfInput['atrCurve'] = curevInd(tfInput['atr'], 'natr', 0, [] if updateArr == [] else updateArr['atrCurve'])
    tfInput['atr2'] = ATR(df, 25, 1, [] if updateArr == [] else updateArr['atr2'])
    tfInput['atr2Slope'] = slopeInd(tfInput['atr2'], 'natr', 1, [] if updateArr == [] else updateArr['atr2Slope'])
    tfInput['atr2Curve'] = curevInd(tfInput['atr2'], 'natr', 1, [] if updateArr == [] else updateArr['atr2Curve'])
    tfInput['atr3'] = ATR(df, 50, 2, [] if updateArr == [] else updateArr['atr3'])
    tfInput['atr3Slope'] = slopeInd(tfInput['atr3'], 'natr', 2, [] if updateArr == [] else updateArr['atr3Slope'])
    tfInput['atr3Curve'] = curevInd(tfInput['atr3'], 'natr', 2, [] if updateArr == [] else updateArr['atr3Curve'])
    #indicators['sentimentPositive'] = sentimentPositive
    #indicators['sentimentNegative'] = sentimentNegative

In [20]:
indi = {}
fillInputs(df, indi)

In [21]:
indicators = copy.deepcopy(indi)
#ierotie = {}
#fillInputs({'open': 28946.0, 'high': 28946.1, 'low': 28945.8, 'close': 28945.9, 'volume': 1,'volPrice':28945.95}, ierotie, indi)

In [22]:
# Inputs to be used for training

relevantInputs = [
    'rsi',
    'rsiSlope',
    'rsiCurve',
    'rsi2',
    'rsi2Slope',
    'rsi2Curve',
    'macd',
    'macdSlope',
    'macdCurve',
    'stOs',
    'stOsSlope',
    'stOsCurve',
    'adx',
    'atr',
    'atrSlope',
    'atrCurve',
    'atr2',
    'atr2Slope',
    'atr2Curve',
    'atr3',
    'atr3Slope',
    'atr3Curve',
    'adxSlope',
    'adxCurve',
    'macd2',
    'macd2Slope',
    'macd2Curve',
    'macd3',
    'macd3Slope',
    'macd3Curve',
    'stOs2',
    'stOs2Slope',
    'stOs2Curve',
    'stOs3',
    'stOs3Slope',
    'stOs3Curve',
    'OindSlope',
    'OindCurve',
    'OindMacd',
    'OindMacdSlope',
    'OindMacdCurve',
    'OindRsi',
    'OindRsiSlope',
    'OindRsiCurve',
    'OindMacd2',
    'OindMacd2Slope',
    'OindMacd2Curve',
    'OindRsi2',
    'OindRsi2Slope',
    'OindRsi2Curve',
    'OindstOs',
    'OindstOsSlope',
    'OindstOsCurve',
    'OindAdx',
    'OindAdxSlope',
    'OindAdxCurve',
    #'sentimentPositive',
    #'sentimentNegative'
]

In [23]:
#Train Data

offsetTrain = 0
slicer = [300, testBreakPoint + offsetTrain]

inputs1 = []
for i in relevantInputs:
    if i in indicators.keys():
        inputs1.append(indicators[i][:])

outputs = outputS[:]

In [24]:
X_train = InputOutputGiver(inputs1[:], slicer, True)
y_train = InputOutputGiver(outputs[:], slicer)

X_train, y_train = shuffle(X_train, y_train)

In [25]:
model = []
scaler = []

In [26]:
print(len(X_train))

1303770


In [27]:
import tensorflow_model_optimization as tfmot
initial_sparsity = 0.0
final_sparsity = 0.9
epochs = 225
batchSize = 256
esFactor = 0.85
begin_step = int(0.1*esFactor*(len(X_train)//batchSize)*epochs)
end_step = int(esFactor*(len(X_train)//batchSize)*epochs)
pruning_params = {
        'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
            initial_sparsity=initial_sparsity,
            final_sparsity=final_sparsity,
            begin_step=begin_step,
            end_step=end_step
        )
    }
pruning_callback = tfmot.sparsity.keras.UpdatePruningStep()

In [28]:
# Model Training

model.append(tf.keras.Sequential([
    tf.keras.layers.Dropout(0.2, input_shape=(len(inputs1),)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(40, activation = tf.keras.activations.relu),
    tf.keras.layers.Dense(25, activation = tf.keras.activations.relu),
    tf.keras.layers.Dense(15, activation = tf.keras.activations.relu),
    tf.keras.layers.Dense(25, activation = tf.keras.activations.relu),
    tf.keras.layers.Dense(40, activation = tf.keras.activations.relu),
    tf.keras.layers.Dense(len(outputS))
]))

model[-1] = tfmot.sparsity.keras.prune_low_magnitude(model[-1], **pruning_params)
model[-1].compile(loss='mse', optimizer=tf.keras.optimizers.experimental.SGD(learning_rate=0.001, momentum = 0.03, use_ema = True), metrics=['mae', 'acc', 'mse'])

scaler.append(preprocessing.StandardScaler())

X_train = scaler[-1].fit_transform(X_train)

model[-1].fit(X_train, y_train, epochs = epochs, batch_size = batchSize, callbacks = pruning_callback)

Epoch 1/225
5093/5093 [==============================] - 10s 1ms/step - loss: 0.3766 - mae: 0.5344 - acc: 0.2396 - mse: 0.3766
Epoch 2/225
5093/5093 [==============================] - 7s 1ms/step - loss: 0.3636 - mae: 0.5238 - acc: 0.2616 - mse: 0.3636
Epoch 3/225
5093/5093 [==============================] - 7s 1ms/step - loss: 0.3625 - mae: 0.5226 - acc: 0.2642 - mse: 0.3625
Epoch 4/225
5093/5093 [==============================] - 8s 2ms/step - loss: 0.3620 - mae: 0.5222 - acc: 0.2730 - mse: 0.3620
Epoch 5/225
5093/5093 [==============================] - 8s 2ms/step - loss: 0.3617 - mae: 0.5220 - acc: 0.2803 - mse: 0.3617
Epoch 6/225
5093/5093 [==============================] - 8s 1ms/step - loss: 0.3615 - mae: 0.5218 - acc: 0.2857 - mse: 0.3615
Epoch 7/225
5093/5093 [==============================] - 7s 1ms/step - loss: 0.3613 - mae: 0.5217 - acc: 0.2898 - mse: 0.3613
Epoch 8/225
5093/5093 [==============================] - 7s 1ms/step - loss: 0.3612 - mae: 0.5217 - acc: 0.2923 - mse

5093/5093 [==============================] - 7s 1ms/step - loss: 0.3601 - mae: 0.5208 - acc: 0.3249 - mse: 0.3601
Epoch 66/225
5093/5093 [==============================] - 7s 1ms/step - loss: 0.3601 - mae: 0.5209 - acc: 0.3429 - mse: 0.3601
Epoch 67/225
5093/5093 [==============================] - 7s 1ms/step - loss: 0.3600 - mae: 0.5209 - acc: 0.3856 - mse: 0.3600
Epoch 68/225
5093/5093 [==============================] - 8s 1ms/step - loss: 0.3600 - mae: 0.5209 - acc: 0.3379 - mse: 0.3600
Epoch 69/225
5093/5093 [==============================] - 7s 1ms/step - loss: 0.3599 - mae: 0.5208 - acc: 0.2873 - mse: 0.3599
Epoch 70/225
5093/5093 [==============================] - 7s 1ms/step - loss: 0.3599 - mae: 0.5205 - acc: 0.2672 - mse: 0.3599
Epoch 71/225
5093/5093 [==============================] - 7s 1ms/step - loss: 0.3599 - mae: 0.5206 - acc: 0.2529 - mse: 0.3599
Epoch 72/225
5093/5093 [==============================] - 7s 1ms/step - loss: 0.3598 - mae: 0.5206 - acc: 0.2645 - mse: 0.35

5093/5093 [==============================] - 8s 2ms/step - loss: 0.3594 - mae: 0.5202 - acc: 0.4409 - mse: 0.3594
Epoch 130/225
5093/5093 [==============================] - 8s 2ms/step - loss: 0.3594 - mae: 0.5203 - acc: 0.3779 - mse: 0.3594
Epoch 131/225
5093/5093 [==============================] - 8s 2ms/step - loss: 0.3594 - mae: 0.5201 - acc: 0.4444 - mse: 0.3594
Epoch 132/225
5093/5093 [==============================] - 7s 1ms/step - loss: 0.3594 - mae: 0.5201 - acc: 0.4435 - mse: 0.3594
Epoch 133/225
5093/5093 [==============================] - 7s 1ms/step - loss: 0.3594 - mae: 0.5201 - acc: 0.4419 - mse: 0.3594
Epoch 134/225
5093/5093 [==============================] - 7s 1ms/step - loss: 0.3593 - mae: 0.5201 - acc: 0.4414 - mse: 0.3593
Epoch 135/225
5093/5093 [==============================] - 7s 1ms/step - loss: 0.3593 - mae: 0.5201 - acc: 0.4429 - mse: 0.3593
Epoch 136/225
5093/5093 [==============================] - 7s 1ms/step - loss: 0.3593 - mae: 0.5201 - acc: 0.4424 - ms

5093/5093 [==============================] - 7s 1ms/step - loss: 0.3584 - mae: 0.5191 - acc: 0.4327 - mse: 0.3584
Epoch 194/225
5093/5093 [==============================] - 7s 1ms/step - loss: 0.3584 - mae: 0.5191 - acc: 0.4333 - mse: 0.3584
Epoch 195/225
5093/5093 [==============================] - 7s 1ms/step - loss: 0.3583 - mae: 0.5191 - acc: 0.4329 - mse: 0.3583
Epoch 196/225
5093/5093 [==============================] - 7s 1ms/step - loss: 0.3583 - mae: 0.5191 - acc: 0.4323 - mse: 0.3583
Epoch 197/225
5093/5093 [==============================] - 7s 1ms/step - loss: 0.3583 - mae: 0.5191 - acc: 0.4312 - mse: 0.3583
Epoch 198/225
5093/5093 [==============================] - 7s 1ms/step - loss: 0.3583 - mae: 0.5191 - acc: 0.4307 - mse: 0.3583
Epoch 199/225
5093/5093 [==============================] - 7s 1ms/step - loss: 0.3583 - mae: 0.5190 - acc: 0.4303 - mse: 0.3583
Epoch 200/225
5093/5093 [==============================] - 7s 1ms/step - loss: 0.3583 - mae: 0.5190 - acc: 0.4301 - ms

In [32]:
globalTempValueStorer['minOut'] = 1
globalTempValueStorer['maxOut'] = -1

def performance(slicer2, corr, total, aveProfT, threshold = 0.5, increase = 48.0, scalingTest = scaler[-1], model = model, lt = 0.25):
    
    print('Starting: ', slicer2)
    inc = 0.05
    dec = 0.001
    
    lenCurve = (len(outputs[0][0]) + slicer2[0])*len(outputs[0])
    
    X_test = InputOutputGiver(inputs1[:], slicer2)

    y_test = InputOutputGiver(outputs[:], slicer2)

    y_pred1 = model[-1].predict(scalingTest.transform(X_test))

    temp = [[0.0, 0.0, 0.0, 0.0] for a in range(len(outputs))]
    param = [threshold for i in range(len(outputs)//2)]
    
    diffCorrIncorr = [[[0.0, 0.0], [0.0, 0.0]] for a in range(len(outputs))]

    for a in range(len(y_pred1)):
        for b in range(len(outputs)):
            toPrintArray = [str(b), ':', str(round(y_pred1[a][b], 3)), str(round(y_test[a][b], 3)), str(a), '--']
            toPrintArray += [str(len(globalTempValueStorer['outProfReturnStraight'][b][lenCurve + a])), 'st:', str(globalTempValueStorer['outProfReturnStraight'][b][lenCurve + a][-1][1][0]), str(globalTempValueStorer['outProfReturnStraight'][b][lenCurve + a][-1][1][1]), '& cl:', str(globalTempValueStorer['outProfReturnStraight'][b][lenCurve + a][-1][2][0]), str(globalTempValueStorer['outProfReturnStraight'][b][lenCurve + a][-1][2][1]), str(globalTempValueStorer['outProfReturnStraight'][b][lenCurve + a][-1][2][2]), str(globalTempValueStorer['outProfReturnStraight'][b][lenCurve + a][-1][2][3]), 'pa:', str(lenCurve)]
            
            if y_pred1[a][b] > param[b//2]:
                if y_test[a][b] > 0:
                    aveProfT[0][1] += 1
                    aveProfT[0][0] += y_test[a][b]
                    globalTempValueStorer['minOut'] = min(globalTempValueStorer['minOut'], y_test[a][b])
                    temp[b][1] += 1
                    diffCorrIncorr[b][0][1] += 1
                    print('Cor', y_pred1[a][b], param[b//2], " ".join(toPrintArray))
                else:
                    print('WRO', y_pred1[a][b], param[b//2], " ".join(toPrintArray))
                temp[b][0] += 1
                diffCorrIncorr[b][0][0] += 1
                param[b//2] += inc
                    
            elif y_pred1[a][b] < -param[b//2]:
                if y_test[a][b] < 0:
                    aveProfT[1][1] += 1
                    aveProfT[1][0] += y_test[a][b]
                    globalTempValueStorer['maxOut'] = max(globalTempValueStorer['maxOut'], y_test[a][b])
                    temp[b][3] += 1
                    diffCorrIncorr[b][1][1] += 1
                    print('Cor', y_pred1[a][b], param[b//2], " ".join(toPrintArray))
                else:
                    print('WRO', y_pred1[a][b], param[b//2], " ".join(toPrintArray))
                temp[b][2] += 1
                diffCorrIncorr[b][1][0] += 1
                param[b//2] += inc
                
            else:
                if param[b//2] > lt:
                    param[b//2] -= dec

    temper = [0.0, 0.0]

    for a in range(len(outputs)):
        if temp[a][0] != 0:
            if a%2 == 1:
                #print('Prev Correct', temp[a][1], temp[a][0], a, 0)
                temper[0] += temp[a][1]
                temper[1] += temp[a][0]
        if temp[a][2] != 0:
            if a%2 == 0:
                #print('Prev Correct', temp[a][3], temp[a][2], a, 1)
                temper[0] += temp[a][3]
                temper[1] += temp[a][2]
    
    if temper[1] != 0:
        print(slicer2, ' --> ', temper[0]/temper[1], ', Num: ', temper[1])
        corr += temper[0]
        total += temper[1]
    else:
        print(slicer2, 'Num: ', temper[1])
        
    if slicer2[1] <= -400:
        return performance([slicer2[0] + int(increase), slicer2[1] + int(increase)], corr, total, aveProfT)
    else:
        print('Finishing', slicer2)
        return [corr, total, aveProfT]

In [33]:
offsetTest = 0

perf = performance([testBreakPoint + offsetTrain + offsetTest, testBreakPoint + 48 + offsetTrain + offsetTest], 0, 0, [[0.0, 0.0], [0.0, 0.0]])

Starting:  [-4000, -3952]
45/45 [==============================] - 0s 637us/step
Cor 0.2545482 0.24999999999999978 1 : 0.255 0.902 129 -- 2 st: 1676483940000 22885.56 & cl: 1676491560000 23390.0 23444.96 23370.5 pa: 1312770
Cor 0.2514361 0.24999999999999972 1 : 0.251 0.94 159 -- 2 st: 1676485740000 22986.43 & cl: 1676491680000 23507.85 23574.8 23409.15 pa: 1312770
Cor 0.25839862 0.24999999999999967 1 : 0.258 0.989 211 -- 2 st: 1676488860000 23290.59 & cl: 1676492400000 23786.64 23831.8 23722.1 pa: 1312770
Cor 0.2509492 0.2499999999999996 1 : 0.251 0.805 287 -- 2 st: 1676493420000 24104.0 & cl: 1676506320000 24627.1 24662.35 24534.11 pa: 1312770
Cor 0.26040104 0.24999999999999956 1 : 0.26 0.905 454 -- 2 st: 1676503440000 24328.4 & cl: 1676506920000 24829.5 24903.0 24802.0 pa: 1312770
Cor 0.26121798 0.2499999999999995 1 : 0.261 0.409 544 -- 2 st: 1676508840000 24623.7 & cl: 1676563380000 25185.5 25200.0 25124.0 pa: 1312770
Cor 0.2598133 0.24999999999999944 1 : 0.26 0.44 574 -- 2 st: 1676

WRO 0.25596774 0.24999999999999883 1 : 0.256 -0.134 841 -- 2 st: 1676785860000 24727.0 & cl: 1676855580000 23981.0 23984.06 23899.37 pa: 1317090
WRO 0.25616756 0.24999999999999878 1 : 0.256 -0.052 890 -- 2 st: 1676788800000 24656.5 & cl: 1676855640000 23858.91 23979.0 23850.01 pa: 1317090
WRO 0.25994307 0.24999999999999872 1 : 0.26 -0.11 1274 -- 2 st: 1676811840000 24707.4 & cl: 1676855580000 23981.0 23984.06 23899.37 pa: 1317090
[-3856, -3808]  -->  0.3 , Num:  20.0
Starting:  [-3808, -3760]
45/45 [==============================] - 0s 590us/step
[-3808, -3760] Num:  0.0
Starting:  [-3760, -3712]
45/45 [==============================] - 0s 594us/step
[-3760, -3712] Num:  0.0
Starting:  [-3712, -3664]
45/45 [==============================] - 0s 611us/step
[-3712, -3664] Num:  0.0
Starting:  [-3664, -3616]
45/45 [==============================] - 0s 622us/step
[-3664, -3616] Num:  0.0
Starting:  [-3616, -3568]
45/45 [==============================] - 0s 682us/step
[-3616, -3568] Num:  0.

45/45 [==============================] - 0s 628us/step
Cor 0.2506417 0.24999999999999978 1 : 0.251 0.75 201 -- 2 st: 1680117060000 28342.0 & cl: 1680143460000 29063.59 29138.85 28888.0 pa: 1373250
Cor 0.25601268 0.24999999999999972 1 : 0.256 0.616 319 -- 2 st: 1680124140000 28425.4 & cl: 1680143460000 29063.59 29138.85 28888.0 pa: 1373250
Cor 0.2515495 0.24999999999999967 1 : 0.252 0.76 379 -- 2 st: 1680127740000 28368.49 & cl: 1680143460000 29063.59 29138.85 28888.0 pa: 1373250
Cor 0.2568739 0.2499999999999996 1 : 0.257 0.64 409 -- 2 st: 1680129540000 28441.21 & cl: 1680143460000 29063.59 29138.85 28888.0 pa: 1373250
Cor 0.25436875 0.24999999999999956 1 : 0.254 1.002 589 -- 2 st: 1680140340000 28411.79 & cl: 1680143460000 29063.59 29138.85 28888.0 pa: 1373250
WRO 0.2589708 0.2499999999999995 1 : 0.259 -0.827 619 -- 2 st: 1680142140000 29095.0 & cl: 1680145860000 28282.05 28373.33 28072.0 pa: 1373250
WRO 0.257531 0.24999999999999944 1 : 0.258 -0.427 709 -- 2 st: 1680147540000 28525.28 

Cor 0.25104815 0.24999999999999917 1 : 0.251 0.347 1130 -- 2 st: 1681209600000 30128.0 & cl: 1681432740000 30762.68 30816.0 30666.0 pa: 1390530
Cor 0.25443497 0.2499999999999991 1 : 0.254 0.313 1227 -- 2 st: 1681215420000 30169.55 & cl: 1681434240000 30882.0 30882.0 30742.0 pa: 1390530
Cor 0.2584253 0.25499999999999906 1 : 0.258 0.34 1250 -- 2 st: 1681216800000 30135.74 & cl: 1681432740000 30762.68 30816.0 30666.0 pa: 1390530
Cor 0.252454 0.249999999999999 1 : 0.252 0.299 1280 -- 2 st: 1681218600000 30187.1 & cl: 1681434240000 30882.0 30882.0 30742.0 pa: 1390530
Cor 0.25058156 0.24999999999999895 1 : 0.251 0.26 1310 -- 2 st: 1681220400000 30241.81 & cl: 1681436820000 30869.0 30941.21 30815.0 pa: 1390530
[-1408, -1360]  -->  1.0 , Num:  16.0
Starting:  [-1360, -1312]
45/45 [==============================] - 0s 589us/step
Cor 0.25141445 0.24999999999999978 1 : 0.251 0.251 440 -- 2 st: 1681254600000 30254.1 & cl: 1681436820000 30869.0 30941.21 30815.0 pa: 1391970
[-1360, -1312]  -->  1.0 

45/45 [==============================] - 0s 584us/step
Cor 0.25694346 0.24999999999999978 1 : 0.257 0.802 916 -- 2 st: 1682838360000 29289.94 & cl: 1682868600000 29952.4 29968.0 29836.08 pa: 1417890
Cor 0.25511003 0.24999999999999972 1 : 0.255 1.017 1224 -- 2 st: 1682856840000 29237.27 & cl: 1682867880000 29803.71 29900.0 29723.8 pa: 1417890
Cor 0.25259522 0.25099999999999967 1 : 0.253 1.0 1249 -- 2 st: 1682858340000 29276.61 & cl: 1682868600000 29952.4 29968.0 29836.08 pa: 1417890
WRO 0.25182658 0.2499999999999996 1 : 0.252 -0.133 1341 -- 2 st: 1682863860000 29401.31 & cl: 1682905860000 28490.28 28687.0 28360.2 pa: 1417890
[-496, -448]  -->  0.75 , Num:  4.0
Starting:  [-448, -400]
45/45 [==============================] - 0s 588us/step
[-448, -400] Num:  0.0
Starting:  [-400, -352]
45/45 [==============================] - 0s 601us/step
[-400, -352] Num:  0.0
Finishing [-400, -352]


In [34]:
print(perf[0]/perf[1], perf[1])

0.5026455026455027 189.0


In [35]:
perTrade = 0.01
startMon = 5000.0
ratio = -2

for a in range(int(perf[1])):
    if a < perf[0]:
        startMon *= (1 + perTrade)
    else:
        startMon *= (1 + perTrade*ratio)
        
print(startMon)

1926.422884816045


In [36]:
def processNewData(data, prevInd, scalingTest = scaler[-1]):
    # data in the form of a dictionary -> unix, open, high, low, close, volume, volPrice
    # ouput is predicted score
    global globalTempValueStorer
    if int(str(data['unix'])[:9]) - int(str(globalTempValueStorer['lastUnixProcessed'])[:9]) == 6:
        testInputs = {}
        fillInputs(data, testInputs, prevInd)
        nnInput = []
        for i in relevantInputs:
            if i in indicators.keys():
                nnInput.append(testInputs[i])
        prediction = model[-1].predict(scalingTest.transform([nnInput]))
        globalTempValueStorer['lastUnixProcessed'] = int(data['unix'])
        print('Compute finished for unix:', data['unix'])
        return prediction[0]
    else:
        print('Time Unix not in sync!')
        print('Unix given:', data['unix'], 'Prev Unix Processed:', globalTempValueStorer['lastUnixProcessed'])

In [37]:
def getNewPrediction():
    global indicators
    newPrediction = []
    newDataAvailable = fetchData(globalTempValueStorer['lastUnixProcessed'])
    newDataAvailable.to_dict()
    
    if len(newDataAvailable) == 1:
        print('No new data available!')
        return []
        
    if len(newDataAvailable) < 20:
        print('Committing Updates')
        for i in range(1, len(newDataAvailable)):
            newData = {}
            for j in newDataAvailable.keys():
                newData[j] = newDataAvailable[j][i]
            t = processNewData(newData, indicators)
            if t is not None:
                newPrediction.append(t)
        return newPrediction
    else:
        print('Restarting data fetching, new data available for', len(newDataAvailable), 'minutes')
        getDfAndDataset()
        indicators = {}
        fillInputs(df, indicators)
        getNewPrediction()

In [38]:
np = getNewPrediction()

No new data available!


In [39]:
print(np)

[]
